## 说明

* 处理完数据就保存起来，然后把处理数据的代码注释掉



## 保存路径
```
path = '10captcha/images/'
num = 100 # 验证码图片数量

if not os.path.exists(path):
    os.makedirs(path)
```

## 1 目录中的子文件操作
* 获取文件名（经常文件名就是对应标签）

In [6]:
import os


In [ ]:
# 获取文件名（经常文件名就是对应标签）

# 获取文件mfcc特征和对应标签
def extract_features(wav_files):
    inputs = []
    labels = []
    
    for wav_file in tqdm(wav_files): # 循环每个文件
        # 读入音频文件
        audio,fs = librosa.load(wav_file) # audio--文件,fs--采样率 

        # 获取音频mfcc特征
        # [n_steps, n_inputs]  n_steps--几秒音频中分的帧数, n_inputs--特征
        mfccs = np.transpose(librosa.feature.mfcc(y=audio, sr=fs, n_mfcc=FLAGS.n_inputs), [1,0]) 
        inputs.append(mfccs.tolist())
    #获取label
    for wav_file in wav_files:
        label = wav_file.split('/')[-1].split('-')[1] # -分隔得到第2个数字
        labels.append(label) 
    return inputs, np.array(labels, dtype=np.int)


## 2 单文件中内容操作(每行  定位内容)

In [3]:
# 获得训练用的wav文件路径列表  
def get_wav_files(parent_dir,sub_dirs): 
    wav_files = []  
    for l, sub_dir in enumerate(sub_dirs):
        wav_path = os.path.join(parent_dir, sub_dir)
        for (dirpath, dirnames, filenames) in os.walk(wav_path):  
            for filename in filenames:  
                if filename.endswith('.wav') or filename.endswith('.WAV'):   # 判断后缀名是否为wav WAV
                    filename_path = os.sep.join([dirpath, filename])  
                    wav_files.append(filename_path)  
    return wav_files  

# 获取文件mfcc特征和对应标签
def extract_features(wav_files):
    inputs = []
    labels = []
    
    for wav_file in tqdm(wav_files):
        # 读入音频文件
        audio,fs = librosa.load(wav_file)

        # 获取音频mfcc特征
        # [n_steps, n_inputs]
        mfccs = np.transpose(librosa.feature.mfcc(y=audio, sr=fs, n_mfcc=FLAGS.n_inputs), [1,0]) 
        inputs.append(mfccs.tolist()) 
    #获取label
    for wav_file in wav_files:
        label = wav_file.split('/')[-1].split('-')[1]
        labels.append(label) 
    return inputs, np.array(labels, dtype=np.int)

In [ ]:
# 获得训练用的wav文件路径列表 
wav_files = get_wav_files(FLAGS.parent_dir,FLAGS.tr_sub_dirs)
# 获取文件mfcc特征和对应标签
tr_features,tr_labels = extract_features(wav_files)

np.save('tr_features.npy',tr_features)
np.save('tr_labels.npy',tr_labels)

# tr_features=np.load('tr_features.npy')
# tr_labels=np.load('tr_labels.npy')